# Medallion Architecture Lab: Bronze → Silver → Gold 🥉🥈🥇

Welcome to the Medallion Architecture lab! In this hands-on lab, you'll build a complete data pipeline using Databricks SQL.

---

## 🎯 Learning Objectives

By the end of this lab, you will be able to:

1. ✅ Understand the medallion architecture (bronze, silver, gold)
2. ✅ Use **COPY INTO** to ingest raw data into bronze tables
3. ✅ Use **MERGE INTO** to clean and deduplicate data for silver tables
4. ✅ Create aggregated business metrics in gold tables
5. ✅ Build an end-to-end data pipeline using SQL
6. ✅ Apply data quality checks and transformations

---

## 🏗️ What is Medallion Architecture?

The **Medallion Architecture** organizes data into three layers:

### **🥉 Bronze Layer (Raw)**
* **Purpose:** Ingest raw data (as-is)
* **Characteristics:** Minimal processing, append-only, full history
* **Quality:** May have duplicates, nulls, bad data
* **Method:** COPY INTO for incremental ingestion

### **🥈 Silver Layer (Cleaned)**
* **Purpose:** Clean, validate, and deduplicate
* **Characteristics:** Business-ready, no duplicates, validated
* **Quality:** High quality, consistent schema
* **Method:** MERGE INTO for upserts and deduplication

### **🥇 Gold Layer (Aggregated)**
* **Purpose:** Business-level aggregations and metrics
* **Characteristics:** Optimized for analytics, pre-aggregated
* **Quality:** Report-ready, fast queries
* **Method:** Aggregation queries, materialized views

---

## 📊 Lab Scenario: E-Commerce Orders

You're building a data pipeline for an e-commerce company:

**Raw data:** Order files arrive in cloud storage (CSV format)

**Your pipeline:**
1. **Bronze:** Ingest raw order files
2. **Silver:** Clean data, remove duplicates, validate
3. **Gold:** Create daily sales metrics for dashboards

---

## 🛠️ Lab Structure

This lab has **10 tasks** to complete:

**Setup (Tasks 1-2):**
1. Create volumes and generate sample order files
2. Explore the raw data

**Bronze Layer (Tasks 3-4):**
3. Create bronze table
4. Use COPY INTO to ingest raw data

**Silver Layer (Tasks 5-6):**
5. Create silver table with data quality rules
6. Use MERGE INTO to clean and deduplicate

**Gold Layer (Tasks 7-8):**
7. Create gold table with daily metrics
8. Build aggregation query

**Validation (Tasks 9-10):**
9. Verify data quality across layers
10. Test incremental updates

**Each task includes:**
* 📝 Clear instructions
* 💡 Hints to guide you
* ✅ Solutions at the end (try first!)

---

**Let's get started!** 🚀

## Task 1: Setup - Create Volume and Generate Raw Data 🛠️

**Your Challenge:**

Create a Unity Catalog volume and generate sample order files to simulate raw data arriving from an e-commerce system.

**Requirements:**

**Part A: Create Volume**
1. Create a volume: `main.default.ecommerce_raw_data`
2. This will store your raw CSV files

**Part B: Generate Sample Order Files**
1. Create 2 batches of order data (simulating data arriving at different times)
2. Each batch should be a CSV file with these columns:
   * `order_id` - INT
   * `customer_id` - INT
   * `order_date` - STRING (YYYY-MM-DD format)
   * `product_name` - STRING
   * `quantity` - INT
   * `unit_price` - DOUBLE
   * `status` - STRING ('completed', 'pending', 'cancelled')

**Batch 1:** 100 orders (order_id 1-100)  
**Batch 2:** 50 orders (order_id 101-150), **including 5 duplicates** from Batch 1 (order_id 1-5)

**Data quality issues to include:**
* Some null values in product_name
* Some negative quantities (data errors)
* Duplicates in Batch 2

---

**Write your code in the cell below:**

In [0]:
# TODO: Create volume and generate 2 batches of CSV files
# Batch 1: 100 orders
# Batch 2: 50 orders + 5 duplicates from Batch 1
# Include data quality issues (nulls, negatives, duplicates)



### 💡 Hints for Task 1

<details>
<summary><b>Hint 1:</b> Create volume (click to expand)</summary>

```python
spark.sql("""
  CREATE VOLUME IF NOT EXISTS main.default.ecommerce_raw_data
  COMMENT 'Raw order files for medallion architecture lab'
""")
```
</details>

<details>
<summary><b>Hint 2:</b> Generate sample data (click to expand)</summary>

```python
import random
from datetime import datetime, timedelta

products = ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones', None]  # Include None for nulls
statuses = ['completed', 'pending', 'cancelled']

data_batch1 = [
    (i, 
     random.randint(1, 50),
     (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
     random.choice(products),
     random.randint(-2, 10),  # Include negative for errors
     round(random.uniform(10, 500), 2),
     random.choice(statuses))
    for i in range(1, 101)
]
```
</details>

<details>
<summary><b>Hint 3:</b> Write CSV files (click to expand)</summary>

```python
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema = StructType([
    StructField("order_id", IntegerType()),
    StructField("customer_id", IntegerType()),
    StructField("order_date", StringType()),
    StructField("product_name", StringType()),
    StructField("quantity", IntegerType()),
    StructField("unit_price", DoubleType()),
    StructField("status", StringType())
])

df = spark.createDataFrame(data, schema)
df.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/main/default/ecommerce_raw_data/batch1")
```
</details>

<details>
<summary><b>Hint 4:</b> Create duplicates in Batch 2 (click to expand)</summary>

```python
# Batch 2: New orders + duplicates from Batch 1
data_batch2 = data_batch1[0:5]  # First 5 orders (duplicates)
data_batch2 += [(i, ...) for i in range(101, 151)]  # New orders
```
</details>

## Task 2: Explore the Raw Data 🔍

**Your Challenge:**

Examine the raw CSV files you created to understand the data quality issues.

**Requirements:**

1. Use `read_files()` to query the CSV files directly
2. Look for:
   * Null values in product_name
   * Negative quantities
   * Duplicate order_ids (between batch1 and batch2)
3. Count total rows across both batches

**Questions to answer:**
* How many total rows are in the raw files?
* How many rows have null product_name?
* How many rows have negative quantity?
* Are there duplicate order_ids?

**Syntax:**
```sql
SELECT * FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
```

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Use read_files() to explore the raw CSV data
-- Check for nulls, negatives, and duplicates



### 💡 Hints for Task 2

<details>
<summary><b>Hint 1:</b> Read all CSV files (click to expand)</summary>

```sql
SELECT * 
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
LIMIT 20
```
</details>

<details>
<summary><b>Hint 2:</b> Count data quality issues (click to expand)</summary>

```sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(CASE WHEN product_name IS NULL THEN 1 END) AS null_products,
  COUNT(CASE WHEN quantity < 0 THEN 1 END) AS negative_quantities
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
```
</details>

<details>
<summary><b>Hint 3:</b> Find duplicates (click to expand)</summary>

```sql
SELECT 
  order_id,
  COUNT(*) AS occurrence_count
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
GROUP BY order_id
HAVING COUNT(*) > 1
ORDER BY order_id
```
</details>

---

# 🥉 Bronze Layer: Raw Data Ingestion

The bronze layer stores raw data exactly as it arrives - no cleaning, no transformations.

**Characteristics:**
* Append-only (keep all data)
* Minimal processing
* May contain duplicates and errors
* Full audit trail
* Uses COPY INTO for incremental loading

---

## Task 3: Create Bronze Table 🥉

**Your Challenge:**

Create a Delta table for the bronze layer.

**Requirements:**

1. Table name: `main.default.orders_bronze`
2. Columns (match the CSV structure):
   * `order_id` INT
   * `customer_id` INT
   * `order_date` STRING (we'll convert to DATE in silver)
   * `product_name` STRING
   * `quantity` INT
   * `unit_price` DOUBLE
   * `status` STRING
   * `ingestion_timestamp` TIMESTAMP (add this for tracking)
3. Use Delta format
4. Add a comment describing the table

**Bronze layer principle:** Store raw data as-is, add metadata columns for tracking.

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Create the bronze table
-- Include all columns from CSV plus ingestion_timestamp



### 💡 Hints for Task 3

<details>
<summary><b>Hint 1:</b> CREATE TABLE syntax (click to expand)</summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.orders_bronze (
  column_name data_type,
  ...
)
USING DELTA
COMMENT 'description'
```
</details>

<details>
<summary><b>Hint 2:</b> All columns needed (click to expand)</summary>

```sql
order_id INT,
customer_id INT,
order_date STRING,
product_name STRING,
quantity INT,
unit_price DOUBLE,
status STRING,
ingestion_timestamp TIMESTAMP
```
</details>

<details>
<summary><b>Hint 3:</b> Why STRING for order_date? (click to expand)</summary>

In bronze, we keep data as-is from the source.
* Source has dates as strings
* We'll convert to proper DATE type in silver layer
* This preserves raw data exactly as received
</details>

## Task 4: Ingest Data with COPY INTO 📥

**Your Challenge:**

Use COPY INTO to incrementally load data from CSV files into the bronze table.

**Requirements:**

**Part A: Load Batch 1**
1. Use COPY INTO to load from `/Volumes/main/default/ecommerce_raw_data/batch1/`
2. Set FORMAT_OPTIONS for CSV (header = true)
3. The ingestion_timestamp should be set to CURRENT_TIMESTAMP()

**Part B: Load Batch 2**
1. Run COPY INTO again for batch2
2. Verify it only loads new files (idempotency)
3. Check total row count

**Expected results:**
* After Batch 1: 100 rows
* After Batch 2: 155 rows (100 + 50 + 5 duplicates)

**COPY INTO syntax:**
```sql
COPY INTO target_table
FROM 'source_path'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
```

---

**Write your code in the cells below:**

In [0]:
%sql
-- TODO: Use COPY INTO to load batch1
-- Don't forget to set ingestion_timestamp to CURRENT_TIMESTAMP()



In [0]:
%sql
-- TODO: Use COPY INTO to load batch2



In [0]:
%sql
-- TODO: Check the bronze table
-- Count total rows, check for duplicates and data quality issues



### 💡 Hints for Task 4

<details>
<summary><b>Hint 1:</b> COPY INTO with column mapping (click to expand)</summary>

```sql
COPY INTO main.default.orders_bronze
FROM (
  SELECT 
    *,
    CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM '/Volumes/main/default/ecommerce_raw_data/batch1/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
```
</details>

<details>
<summary><b>Hint 2:</b> Load batch2 (click to expand)</summary>

```sql
COPY INTO main.default.orders_bronze
FROM (
  SELECT 
    *,
    CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM '/Volumes/main/default/ecommerce_raw_data/batch2/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
```
</details>

<details>
<summary><b>Hint 3:</b> Verify data (click to expand)</summary>

```sql
-- Count total rows
SELECT COUNT(*) AS total_rows FROM main.default.orders_bronze

-- Check for issues
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicate_count,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN quantity < 0 THEN 1 ELSE 0 END) AS negative_quantities
FROM main.default.orders_bronze
```
</details>

---

# 🥈 Silver Layer: Cleaned & Validated Data

The silver layer contains cleaned, validated, and deduplicated data ready for business use.

**Transformations:**
* Remove duplicates (keep latest version)
* Filter out invalid data (nulls, negatives)
* Convert data types (STRING → DATE)
* Add calculated columns (total_amount)
* Enforce data quality rules

**Method:** MERGE INTO for upserts and deduplication

---

## Task 5: Create Silver Table 🥈

**Your Challenge:**

Create a cleaned version of the orders table with proper data types and calculated columns.

**Requirements:**

1. Table name: `main.default.orders_silver`
2. Columns:
   * `order_id` INT (primary key)
   * `customer_id` INT
   * `order_date` DATE (converted from STRING)
   * `product_name` STRING
   * `quantity` INT
   * `unit_price` DOUBLE
   * `total_amount` DOUBLE (calculated: quantity * unit_price)
   * `status` STRING
   * `created_at` TIMESTAMP (when first inserted)
   * `updated_at` TIMESTAMP (when last updated)
3. Use Delta format
4. Add comment: 'Cleaned and validated orders - Silver layer'

**Key differences from bronze:**
* order_date is DATE (not STRING)
* Added total_amount (calculated column)
* Added created_at and updated_at for tracking

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Create the silver table with proper data types
-- Include calculated column (total_amount)



### 💡 Hints for Task 5

<details>
<summary><b>Hint 1:</b> Complete CREATE TABLE (click to expand)</summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.orders_silver (
  order_id INT,
  customer_id INT,
  order_date DATE,
  product_name STRING,
  quantity INT,
  unit_price DOUBLE,
  total_amount DOUBLE,
  status STRING,
  created_at TIMESTAMP,
  updated_at TIMESTAMP
)
USING DELTA
COMMENT 'Cleaned and validated orders - Silver layer'
```
</details>

<details>
<summary><b>Hint 2:</b> Why these columns? (click to expand)</summary>

* `order_date DATE` - Proper type for date operations
* `total_amount` - Pre-calculated for performance
* `created_at` - Track when order first appeared
* `updated_at` - Track when order was last modified
</details>

## Task 6: Clean and Load Silver with MERGE INTO ✨

**Your Challenge:**

Use MERGE INTO to load cleaned data from bronze to silver.

**Requirements:**

**Data Quality Rules:**
1. **Remove duplicates** - Keep only one row per order_id (latest by ingestion_timestamp)
2. **Filter out invalid data:**
   * Skip rows where product_name IS NULL
   * Skip rows where quantity <= 0
   * Skip rows where unit_price <= 0
3. **Transform data:**
   * Convert order_date from STRING to DATE
   * Calculate total_amount = quantity * unit_price
4. **Use MERGE INTO:**
   * WHEN MATCHED: Update existing orders
   * WHEN NOT MATCHED: Insert new orders

**Steps:**
1. Create a CTE to clean and deduplicate bronze data
2. Use MERGE INTO to upsert into silver
3. Set created_at on INSERT, updated_at on both INSERT and UPDATE

**Expected results:**
* Should have ~145 rows (155 minus ~10 invalid rows)
* No duplicates (order_id is unique)
* All data quality rules applied

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Create CTE to clean bronze data, then MERGE into silver
-- Apply all data quality rules
-- Handle duplicates (keep latest by ingestion_timestamp)



In [0]:
%sql
-- TODO: Verify silver table
-- Check row count, no duplicates, no invalid data



### 💡 Hints for Task 6

<details>
<summary><b>Hint 1:</b> Clean and deduplicate CTE (click to expand)</summary>

```sql
WITH cleaned_orders AS (
  SELECT 
    order_id,
    customer_id,
    CAST(order_date AS DATE) AS order_date,
    product_name,
    quantity,
    unit_price,
    quantity * unit_price AS total_amount,
    status,
    ingestion_timestamp,
    ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY ingestion_timestamp DESC) AS rn
  FROM main.default.orders_bronze
  WHERE product_name IS NOT NULL
    AND quantity > 0
    AND unit_price > 0
)
SELECT * FROM cleaned_orders WHERE rn = 1
```
</details>

<details>
<summary><b>Hint 2:</b> MERGE INTO structure (click to expand)</summary>

```sql
MERGE INTO main.default.orders_silver AS target
USING cleaned_orders AS source
ON target.order_id = source.order_id

WHEN MATCHED THEN
  UPDATE SET
    target.customer_id = source.customer_id,
    ...,
    target.updated_at = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
  INSERT (...)
  VALUES (..., CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP())
```
</details>

<details>
<summary><b>Hint 3:</b> Complete solution structure (click to expand)</summary>

You need:
1. CTE to clean and deduplicate
2. MERGE INTO with both WHEN MATCHED and WHEN NOT MATCHED
3. Set created_at and updated_at appropriately
4. Don't include the rn column in the MERGE
</details>

---

# 🥇 Gold Layer: Business Metrics

The gold layer contains aggregated, business-ready data optimized for analytics and reporting.

**Characteristics:**
* Pre-aggregated metrics
* Optimized for dashboards
* Fast query performance
* Business-friendly column names
* Often materialized views or summary tables

**Common patterns:**
* Daily/monthly aggregations
* Customer metrics
* Product performance
* KPIs and business metrics

---

## Task 7: Create Gold Table 🥇

**Your Challenge:**

Create a gold table for daily sales metrics.

**Requirements:**

1. Table name: `main.default.daily_sales_gold`
2. Columns:
   * `order_date` DATE
   * `total_orders` INT
   * `total_revenue` DOUBLE
   * `avg_order_value` DOUBLE
   * `total_quantity_sold` INT
   * `unique_customers` INT
   * `completed_orders` INT
   * `cancelled_orders` INT
   * `updated_at` TIMESTAMP
3. Use Delta format
4. Add comment: 'Daily sales metrics - Gold layer'

**This table will store one row per date with aggregated metrics.**

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Create the gold table for daily metrics



### 💡 Hints for Task 7

<details>
<summary><b>Hint 1:</b> CREATE TABLE syntax (click to expand)</summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.daily_sales_gold (
  order_date DATE,
  total_orders INT,
  total_revenue DOUBLE,
  avg_order_value DOUBLE,
  total_quantity_sold INT,
  unique_customers INT,
  completed_orders INT,
  cancelled_orders INT,
  updated_at TIMESTAMP
)
USING DELTA
COMMENT 'Daily sales metrics - Gold layer'
```
</details>

<details>
<summary><b>Hint 2:</b> Why these metrics? (click to expand)</summary>

These are common business metrics:
* `total_orders` - Volume metric
* `total_revenue` - Financial metric
* `avg_order_value` - Performance metric
* `unique_customers` - Customer metric
* `completed_orders` / `cancelled_orders` - Status metrics
</details>

## Task 8: Populate Gold Table with Aggregations 📈

**Your Challenge:**

Write a query to calculate daily metrics from silver and insert into gold.

**Requirements:**

1. Query the silver table
2. Group by order_date
3. Calculate these metrics:
   * `total_orders` - COUNT(*)
   * `total_revenue` - SUM(total_amount)
   * `avg_order_value` - AVG(total_amount)
   * `total_quantity_sold` - SUM(quantity)
   * `unique_customers` - COUNT(DISTINCT customer_id)
   * `completed_orders` - COUNT where status = 'completed'
   * `cancelled_orders` - COUNT where status = 'cancelled'
4. Use INSERT OVERWRITE to populate the gold table
5. Set updated_at to CURRENT_TIMESTAMP()

**Hint:** Use CASE WHEN for conditional counts.

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Write aggregation query and INSERT into gold table
-- Group by order_date and calculate all metrics



In [0]:
%sql
-- TODO: Query the gold table to see daily metrics
-- Order by date to see trends



### 💡 Hints for Task 8

<details>
<summary><b>Hint 1:</b> Aggregation query structure (click to expand)</summary>

```sql
SELECT 
  order_date,
  COUNT(*) AS total_orders,
  SUM(total_amount) AS total_revenue,
  AVG(total_amount) AS avg_order_value,
  SUM(quantity) AS total_quantity_sold,
  COUNT(DISTINCT customer_id) AS unique_customers,
  SUM(CASE WHEN status = 'completed' THEN 1 ELSE 0 END) AS completed_orders,
  SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
  CURRENT_TIMESTAMP() AS updated_at
FROM main.default.orders_silver
GROUP BY order_date
```
</details>

<details>
<summary><b>Hint 2:</b> INSERT OVERWRITE (click to expand)</summary>

```sql
INSERT OVERWRITE main.default.daily_sales_gold
SELECT 
  -- aggregation query here
FROM main.default.orders_silver
GROUP BY order_date
```
</details>

<details>
<summary><b>Hint 3:</b> Why INSERT OVERWRITE? (click to expand)</summary>

For gold tables with full refresh:
* INSERT OVERWRITE replaces all data
* Recalculates all metrics from silver
* Ensures consistency
* Simple and reliable

Alternatively, you could use MERGE INTO for incremental updates.
</details>

---

# ✅ Validation: Verify Your Pipeline

Let's verify that your medallion architecture pipeline works correctly!

---

## Task 9: Validate Data Quality Across Layers 🔍

**Your Challenge:**

Verify that data quality improves as it flows through the pipeline.

**Requirements:**

Create a query that compares all three layers:

1. **Bronze metrics:**
   * Total rows
   * Duplicate count
   * Null product_name count
   * Negative quantity count

2. **Silver metrics:**
   * Total rows (should be less than bronze)
   * Duplicate count (should be 0)
   * Null count (should be 0)
   * Negative count (should be 0)

3. **Gold metrics:**
   * Total rows (number of unique dates)
   * Total revenue sum

**Expected results:**
* Bronze: ~155 rows with issues
* Silver: ~145 rows, clean
* Gold: ~30 rows (one per date)

**Use UNION ALL to combine metrics from all three layers.**

---

**Write your code in the cell below:**

In [0]:
%sql
-- TODO: Create a comparison query showing metrics from all three layers
-- Use UNION ALL to combine results



### 💡 Hints for Task 9

<details>
<summary><b>Hint 1:</b> Query structure (click to expand)</summary>

```sql
SELECT 
  'Bronze' AS layer,
  COUNT(*) AS total_rows,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicates,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products
FROM main.default.orders_bronze

UNION ALL

SELECT 
  'Silver' AS layer,
  COUNT(*) AS total_rows,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicates,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products
FROM main.default.orders_silver

UNION ALL

SELECT 
  'Gold' AS layer,
  COUNT(*) AS total_rows,
  0 AS duplicates,
  0 AS null_products
FROM main.default.daily_sales_gold
```
</details>

<details>
<summary><b>Hint 2:</b> What to verify (click to expand)</summary>

**Bronze should have:**
* More rows than silver
* Duplicates present
* Null values present
* Negative quantities present

**Silver should have:**
* Fewer rows (invalid data removed)
* No duplicates
* No nulls
* No negative values

**Gold should have:**
* Much fewer rows (aggregated by date)
* Summary metrics only
</details>

## Task 10: Test Incremental Updates 🔄

**Your Challenge:**

Test that your pipeline handles new data correctly.

**Requirements:**

**Part A: Add new raw data**
1. Generate a 3rd batch of orders (order_id 151-200)
2. Write to `/Volumes/main/default/ecommerce_raw_data/batch3/`

**Part B: Run the pipeline**
1. Use COPY INTO to load batch3 into bronze
2. Use MERGE INTO to update silver
3. Recalculate gold metrics

**Part C: Verify**
1. Check row counts increased appropriately
2. Verify no duplicates in silver
3. Verify gold metrics updated

**This tests the end-to-end incremental processing!**

---

**Write your code in the cells below:**

In [0]:
# TODO: Generate batch 3 data (50 more orders, order_id 151-200)
# Write to batch3 directory



In [0]:
%sql
-- TODO: Use COPY INTO to load batch3 into bronze



In [0]:
%sql
-- TODO: Run your MERGE INTO query again to update silver



In [0]:
%sql
-- TODO: Recalculate gold metrics (INSERT OVERWRITE)



In [0]:
%sql
-- TODO: Check row counts in all three layers
-- Bronze: should have ~205 rows
-- Silver: should have ~195 rows
-- Gold: should have ~30 rows (dates)



### 💡 Hints for Task 10

<details>
<summary><b>Hint 1:</b> Generate batch 3 (click to expand)</summary>

```python
# Similar to Task 1, but different ID range
data_batch3 = [
    (i, 
     random.randint(1, 50),
     (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
     random.choice(['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones']),
     random.randint(1, 10),
     round(random.uniform(10, 500), 2),
     random.choice(['completed', 'pending', 'cancelled']))
    for i in range(151, 201)
]

df_batch3 = spark.createDataFrame(data_batch3, schema)
df_batch3.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/main/default/ecommerce_raw_data/batch3")
```
</details>

<details>
<summary><b>Hint 2:</b> Rerun the pipeline (click to expand)</summary>

Just run the same commands again:
1. COPY INTO for bronze (loads only batch3)
2. MERGE INTO for silver (processes new bronze data)
3. INSERT OVERWRITE for gold (recalculates all metrics)
</details>

<details>
<summary><b>Hint 3:</b> Verify counts (click to expand)</summary>

```sql
SELECT 'Bronze' AS layer, COUNT(*) AS row_count FROM main.default.orders_bronze
UNION ALL
SELECT 'Silver' AS layer, COUNT(*) AS row_count FROM main.default.orders_silver
UNION ALL
SELECT 'Gold' AS layer, COUNT(*) AS row_count FROM main.default.daily_sales_gold
```
</details>

---
---
---

# 📝 Complete Solutions

**⚠️ Only look at these if you're stuck or want to verify your work!**

Try to solve the challenges yourself first. Learning happens through problem-solving!

---

## ✅ Solution: Task 1 (Setup)

<details>
<summary><b>Click to reveal solution</b></summary>

```python
import random
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Create volume
spark.sql("""
  CREATE VOLUME IF NOT EXISTS main.default.ecommerce_raw_data
  COMMENT 'Raw order files for medallion architecture lab'
""")

print("✅ Volume created")

# Define schema
schema = StructType([
    StructField("order_id", IntegerType()),
    StructField("customer_id", IntegerType()),
    StructField("order_date", StringType()),
    StructField("product_name", StringType()),
    StructField("quantity", IntegerType()),
    StructField("unit_price", DoubleType()),
    StructField("status", StringType())
])

products = ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones', None]  # Include None
statuses = ['completed', 'pending', 'cancelled']

# Batch 1: 100 orders
data_batch1 = [
    (i, 
     random.randint(1, 50),
     (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
     random.choice(products),
     random.randint(-2, 10),  # Include negatives
     round(random.uniform(10, 500), 2),
     random.choice(statuses))
    for i in range(1, 101)
]

df_batch1 = spark.createDataFrame(data_batch1, schema)
df_batch1.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/main/default/ecommerce_raw_data/batch1")
print("✅ Batch 1 created: 100 orders")

# Batch 2: 50 new orders + 5 duplicates
data_batch2 = data_batch1[0:5]  # Duplicates
data_batch2 += [
    (i, 
     random.randint(1, 50),
     (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
     random.choice(products),
     random.randint(-2, 10),
     round(random.uniform(10, 500), 2),
     random.choice(statuses))
    for i in range(101, 151)
]

df_batch2 = spark.createDataFrame(data_batch2, schema)
df_batch2.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/main/default/ecommerce_raw_data/batch2")
print("✅ Batch 2 created: 50 orders + 5 duplicates")

print("\n✅ Setup complete! Raw data files ready.")
```

</details>

## ✅ Solution: Task 2 (Explore Raw Data)

<details>
<summary><b>Click to reveal solution</b></summary>

**View raw data:**
```sql
SELECT * 
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
LIMIT 20
```

**Check data quality issues:**
```sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicates,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN CAST(quantity AS INT) < 0 THEN 1 ELSE 0 END) AS negative_quantities
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
```

**Find duplicate order_ids:**
```sql
SELECT 
  order_id,
  COUNT(*) AS occurrence_count
FROM read_files(
  '/Volumes/main/default/ecommerce_raw_data/',
  format => 'csv',
  header => true
)
GROUP BY order_id
HAVING COUNT(*) > 1
ORDER BY order_id
```

**Expected findings:**
* Total: 155 rows
* Duplicates: 5 (order_id 1-5)
* Nulls: ~10-15 rows
* Negatives: ~10-15 rows

</details>

## ✅ Solution: Task 3 (Create Bronze Table)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.orders_bronze (
  order_id INT,
  customer_id INT,
  order_date STRING,
  product_name STRING,
  quantity INT,
  unit_price DOUBLE,
  status STRING,
  ingestion_timestamp TIMESTAMP
)
USING DELTA
COMMENT 'Raw order data - Bronze layer'
```

**Key points:**
* `order_date` is STRING (raw format from CSV)
* Added `ingestion_timestamp` for tracking
* Using DELTA format for ACID properties
* IF NOT EXISTS prevents errors if table exists

</details>

## ✅ Solution: Task 4 (Ingest with COPY INTO)

<details>
<summary><b>Click to reveal solution</b></summary>

**Load Batch 1:**
```sql
COPY INTO main.default.orders_bronze
FROM (
  SELECT 
    *,
    CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM '/Volumes/main/default/ecommerce_raw_data/batch1/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
```

**Load Batch 2:**
```sql
COPY INTO main.default.orders_bronze
FROM (
  SELECT 
    *,
    CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM '/Volumes/main/default/ecommerce_raw_data/batch2/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
```

**Verify:**
```sql
-- Check row count
SELECT COUNT(*) AS total_rows FROM main.default.orders_bronze

-- Check data quality
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN quantity < 0 THEN 1 ELSE 0 END) AS negative_quantities
FROM main.default.orders_bronze
```

**Key concepts:**
* COPY INTO tracks processed files (idempotent)
* Subquery adds ingestion_timestamp
* inferSchema automatically detects types
* Bronze contains ALL data (including bad data)

</details>

## ✅ Solution: Task 5 (Create Silver Table)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.orders_silver (
  order_id INT,
  customer_id INT,
  order_date DATE,
  product_name STRING,
  quantity INT,
  unit_price DOUBLE,
  total_amount DOUBLE,
  status STRING,
  created_at TIMESTAMP,
  updated_at TIMESTAMP
)
USING DELTA
COMMENT 'Cleaned and validated orders - Silver layer'
```

**Key differences from bronze:**
* `order_date` is DATE (not STRING)
* Added `total_amount` (calculated column)
* Added `created_at` and `updated_at` for tracking
* No ingestion_timestamp (that's bronze metadata)

</details>

## ✅ Solution: Task 6 (Clean and Load Silver)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
MERGE INTO main.default.orders_silver AS target
USING (
  WITH cleaned_orders AS (
    SELECT 
      order_id,
      customer_id,
      CAST(order_date AS DATE) AS order_date,
      product_name,
      quantity,
      unit_price,
      quantity * unit_price AS total_amount,
      status,
      ingestion_timestamp,
      ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY ingestion_timestamp DESC) AS rn
    FROM main.default.orders_bronze
    WHERE product_name IS NOT NULL
      AND quantity > 0
      AND unit_price > 0
  )
  SELECT 
    order_id,
    customer_id,
    order_date,
    product_name,
    quantity,
    unit_price,
    total_amount,
    status
  FROM cleaned_orders
  WHERE rn = 1
) AS source
ON target.order_id = source.order_id

WHEN MATCHED THEN
  UPDATE SET
    target.customer_id = source.customer_id,
    target.order_date = source.order_date,
    target.product_name = source.product_name,
    target.quantity = source.quantity,
    target.unit_price = source.unit_price,
    target.total_amount = source.total_amount,
    target.status = source.status,
    target.updated_at = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
  INSERT (order_id, customer_id, order_date, product_name, quantity, unit_price, total_amount, status, created_at, updated_at)
  VALUES (source.order_id, source.customer_id, source.order_date, source.product_name, source.quantity, source.unit_price, source.total_amount, source.status, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP())
```

**Verify:**
```sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN quantity <= 0 THEN 1 ELSE 0 END) AS invalid_quantities
FROM main.default.orders_silver
```

**What this does:**
1. CTE filters out invalid data (nulls, negatives)
2. ROW_NUMBER deduplicates (keeps latest)
3. CAST converts order_date to DATE
4. Calculates total_amount
5. MERGE upserts into silver
6. Sets created_at on INSERT, updated_at on both

**Expected results:**
* ~145 rows (155 minus ~10 invalid)
* 0 duplicates
* 0 nulls
* 0 negative quantities

</details>

## ✅ Solution: Task 7 (Create Gold Table)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
CREATE TABLE IF NOT EXISTS main.default.daily_sales_gold (
  order_date DATE,
  total_orders INT,
  total_revenue DOUBLE,
  avg_order_value DOUBLE,
  total_quantity_sold INT,
  unique_customers INT,
  completed_orders INT,
  cancelled_orders INT,
  updated_at TIMESTAMP
)
USING DELTA
COMMENT 'Daily sales metrics - Gold layer'
```

**Key points:**
* One row per order_date
* All metrics are aggregated
* Business-friendly column names
* Ready for dashboards and reports

</details>

## ✅ Solution: Task 8 (Populate Gold Table)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
INSERT OVERWRITE main.default.daily_sales_gold
SELECT 
  order_date,
  COUNT(*) AS total_orders,
  ROUND(SUM(total_amount), 2) AS total_revenue,
  ROUND(AVG(total_amount), 2) AS avg_order_value,
  SUM(quantity) AS total_quantity_sold,
  COUNT(DISTINCT customer_id) AS unique_customers,
  SUM(CASE WHEN status = 'completed' THEN 1 ELSE 0 END) AS completed_orders,
  SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled_orders,
  CURRENT_TIMESTAMP() AS updated_at
FROM main.default.orders_silver
GROUP BY order_date
ORDER BY order_date
```

**Verify:**
```sql
SELECT * 
FROM main.default.daily_sales_gold
ORDER BY order_date
```

**What this does:**
1. Groups by order_date
2. Calculates all business metrics
3. Uses CASE WHEN for conditional counts
4. ROUND for clean numbers
5. INSERT OVERWRITE replaces all data (full refresh)

**Expected results:**
* ~30 rows (one per unique date)
* Each row has aggregated metrics for that day
* Ready for dashboard visualization

</details>

## ✅ Solution: Task 9 (Data Quality Validation)

<details>
<summary><b>Click to reveal solution</b></summary>

```sql
SELECT 
  'Bronze' AS layer,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicates,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN quantity < 0 THEN 1 ELSE 0 END) AS negative_quantities
FROM main.default.orders_bronze

UNION ALL

SELECT 
  'Silver' AS layer,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS unique_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS duplicates,
  SUM(CASE WHEN product_name IS NULL THEN 1 ELSE 0 END) AS null_products,
  SUM(CASE WHEN quantity < 0 THEN 1 ELSE 0 END) AS negative_quantities
FROM main.default.orders_silver

UNION ALL

SELECT 
  'Gold' AS layer,
  COUNT(*) AS total_rows,
  0 AS unique_orders,
  0 AS duplicates,
  0 AS null_products,
  0 AS negative_quantities
FROM main.default.daily_sales_gold

ORDER BY layer
```

**Expected results:**

| layer | total_rows | unique_orders | duplicates | null_products | negative_quantities |
|-------|------------|---------------|------------|---------------|--------------------|
| Bronze | 155 | 150 | 5 | ~10 | ~10 |
| Silver | ~145 | ~145 | 0 | 0 | 0 |
| Gold | ~30 | 0 | 0 | 0 | 0 |

**Key insights:**
* Bronze has all issues (duplicates, nulls, negatives)
* Silver is clean (all issues removed)
* Gold is aggregated (much fewer rows)

</details>

## ✅ Solution: Task 10 (Test Incremental Updates)

<details>
<summary><b>Click to reveal solution</b></summary>

**Generate Batch 3:**
```python
# Generate 50 more orders
data_batch3 = [
    (i, 
     random.randint(1, 50),
     (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
     random.choice(['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones']),
     random.randint(1, 10),
     round(random.uniform(10, 500), 2),
     random.choice(['completed', 'pending', 'cancelled']))
    for i in range(151, 201)
]

df_batch3 = spark.createDataFrame(data_batch3, schema)
df_batch3.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/main/default/ecommerce_raw_data/batch3")
print("✅ Batch 3 created: 50 orders")
```

**Load to Bronze:**
```sql
COPY INTO main.default.orders_bronze
FROM (
  SELECT 
    *,
    CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM '/Volumes/main/default/ecommerce_raw_data/batch3/'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
```

**Update Silver (rerun Task 6 MERGE):**
```sql
-- Same MERGE query from Task 6
MERGE INTO main.default.orders_silver AS target
USING (
  -- Same cleaning logic
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN UPDATE SET ...
WHEN NOT MATCHED THEN INSERT ...
```

**Update Gold (rerun Task 8):**
```sql
-- Same aggregation query from Task 8
INSERT OVERWRITE main.default.daily_sales_gold
SELECT 
  order_date,
  COUNT(*) AS total_orders,
  -- ... all metrics
FROM main.default.orders_silver
GROUP BY order_date
```

**Verify:**
```sql
SELECT 
  'Bronze' AS layer, 
  COUNT(*) AS row_count 
FROM main.default.orders_bronze

UNION ALL

SELECT 
  'Silver' AS layer, 
  COUNT(*) AS row_count 
FROM main.default.orders_silver

UNION ALL

SELECT 
  'Gold' AS layer, 
  COUNT(*) AS row_count 
FROM main.default.daily_sales_gold
```

**Expected results:**
* Bronze: ~205 rows (155 + 50)
* Silver: ~195 rows (cleaned)
* Gold: ~30 rows (dates)

**Key insight:** The pipeline is reusable - just rerun the same queries for new data!

</details>

## 📚 Medallion Architecture Best Practices

### **🥉 Bronze Layer**

✅ **Keep raw data as-is** - No transformations  
✅ **Append-only** - Never delete from bronze  
✅ **Add metadata columns** - ingestion_timestamp, source_file  
✅ **Use COPY INTO** - Incremental, idempotent loading  
✅ **Partition by date** - If data has time dimension  

**Purpose:** Audit trail, reprocessing capability, data lineage

---

### **🥈 Silver Layer**

✅ **Apply data quality rules** - Filter nulls, validate ranges  
✅ **Deduplicate** - One row per business key  
✅ **Convert data types** - STRING → DATE, proper types  
✅ **Add calculated columns** - Derived values  
✅ **Use MERGE INTO** - Upserts, handle updates  
✅ **Track lineage** - created_at, updated_at  

**Purpose:** Clean, validated data for analytics

---

### **🥇 Gold Layer**

✅ **Pre-aggregate** - Calculate metrics once  
✅ **Business-friendly** - Clear column names  
✅ **Optimize for queries** - Denormalize if needed  
✅ **Use INSERT OVERWRITE or MERGE** - Depends on pattern  
✅ **Document metrics** - What each column means  

**Purpose:** Fast dashboards, reports, analytics

---

### **General Best Practices**

✅ **Separate concerns** - Each layer has clear purpose  
✅ **Idempotent pipelines** - Safe to rerun  
✅ **Incremental processing** - Process only new data  
✅ **Monitor data quality** - Track metrics at each layer  
✅ **Use Delta Lake** - ACID, time travel, performance  
✅ **Schedule appropriately** - Bronze (frequent), Silver (hourly), Gold (daily)  
✅ **Test with small data** - Validate logic before production  

## 🔄 Common Pipeline Patterns

### **Pattern 1: Batch Pipeline (This Lab)**

```
Raw Files → Bronze (COPY INTO) → Silver (MERGE) → Gold (INSERT OVERWRITE)
```

**Schedule:**
* Bronze: Every hour (ingest new files)
* Silver: Every hour (clean new data)
* Gold: Daily (aggregate for reports)

---

### **Pattern 2: Streaming Pipeline**

```
Raw Files → Bronze (Auto Loader) → Silver (Stream) → Gold (Stream)
```

**Use when:**
* Need real-time data
* Continuous file arrival
* Low latency requirements

---

### **Pattern 3: CDC Pipeline**

```
Change Data → Bronze (COPY INTO) → Silver (MERGE with SCD) → Gold (Aggregations)
```

**Use when:**
* Capturing database changes
* Need historical tracking
* SCD Type 2 requirements

---

### **Pattern 4: Multi-Source Pipeline**

```
Source A → Bronze A →
                        → Silver (JOIN) → Gold
Source B → Bronze B →
```

**Use when:**
* Multiple data sources
* Need to join data
* Different ingestion schedules

---

### **This Lab's Pipeline**

```
CSV Files in Volume
    ↓
    COPY INTO (incremental)
    ↓
🥉 Bronze: orders_bronze (155 rows, raw data)
    ↓
    MERGE INTO (clean, dedupe)
    ↓
🥈 Silver: orders_silver (~145 rows, validated)
    ↓
    INSERT OVERWRITE (aggregate)
    ↓
🥇 Gold: daily_sales_gold (~30 rows, metrics)
    ↓
    Dashboards & Reports
```

## 💡 Key Concepts Summary

### **Medallion Architecture**

**Bronze → Silver → Gold** = **Raw → Cleaned → Aggregated**

**Why use it?**
* Clear separation of concerns
* Incremental processing
* Data quality improvement
* Reprocessing capability
* Audit trail

---

### **COPY INTO (Bronze)**

**Purpose:** Incremental file ingestion

**Key features:**
* Idempotent (safe to rerun)
* Tracks processed files
* SQL-based
* Works in serverless

**When to use:**
* Loading files from cloud storage
* Batch ingestion
* Bronze layer ingestion

---

### **MERGE INTO (Silver)**

**Purpose:** Upsert and deduplication

**Key features:**
* Atomic operation
* Handles updates and inserts
* No duplicates
* Supports complex logic

**When to use:**
* Cleaning and validating data
* Deduplication
* SCD patterns
* Silver layer updates

---

### **Aggregations (Gold)**

**Purpose:** Business metrics

**Key features:**
* Pre-calculated metrics
* Fast queries
* Dashboard-ready
* Business-friendly

**When to use:**
* Creating KPIs
* Dashboard data
* Report tables
* Gold layer metrics

---

### **Data Quality**

**Bronze:** Accept all data (good and bad)  
**Silver:** Filter and validate  
**Gold:** Aggregate clean data  

**Quality improves at each layer!**

## 🎉 Lab Complete!

Congratulations! You've successfully built a complete medallion architecture pipeline!

### **What You Accomplished:**

✅ **Created 3-layer architecture** - Bronze, Silver, Gold  
✅ **Used COPY INTO** - Incremental file ingestion  
✅ **Used MERGE INTO** - Data cleaning and deduplication  
✅ **Applied data quality rules** - Filtered invalid data  
✅ **Created business metrics** - Aggregated gold layer  
✅ **Tested incremental updates** - End-to-end pipeline  
✅ **Used Unity Catalog** - Modern data governance  
✅ **Pure SQL pipeline** - Works in serverless  

---

### **Your Pipeline:**

```
📁 Raw CSV Files (155 rows with issues)
         ↓
      COPY INTO
         ↓
🥉 Bronze Layer (155 rows, raw)
         ↓
      MERGE INTO (clean, dedupe)
         ↓
🥈 Silver Layer (~145 rows, validated)
         ↓
      INSERT OVERWRITE (aggregate)
         ↓
🥇 Gold Layer (~30 rows, metrics)
         ↓
    📊 Dashboards
```

---

### **Key Takeaways:**

1. **Medallion architecture organizes data** - Clear layers with purpose
2. **Bronze preserves raw data** - Full audit trail
3. **Silver ensures quality** - Clean, validated, deduplicated
4. **Gold optimizes for analytics** - Pre-aggregated metrics
5. **COPY INTO for ingestion** - Incremental, idempotent
6. **MERGE INTO for cleaning** - Upserts without duplicates
7. **Each layer adds value** - Progressive refinement

---

### **Production Checklist:**

☐ Schedule bronze ingestion (hourly/daily)  
☐ Schedule silver processing (after bronze)  
☐ Schedule gold aggregation (daily)  
☐ Add data quality monitoring  
☐ Set up alerts for failures  
☐ Document data lineage  
☐ Implement error handling  
☐ Add data validation tests  

---

### **Next Steps:**

* Build medallion pipelines for your data
* Explore Delta Live Tables (DLT) for declarative pipelines
* Add data quality expectations
* Implement Change Data Capture (CDC)
* Create dashboards from gold tables
* Learn about data mesh architecture

---

### **Resources:**

* [Medallion Architecture Guide](https://www.databricks.com/glossary/medallion-architecture)
* [Delta Lake Best Practices](https://docs.databricks.com/delta/best-practices.html)
* [COPY INTO Documentation](https://docs.databricks.com/sql/language-manual/delta-copy-into.html)
* [MERGE INTO Documentation](https://docs.databricks.com/sql/language-manual/delta-merge-into.html)

---

**You're now ready to build production data pipelines!** 🚀

*Happy data engineering!*